# Actions and Orbit caluclation with MC sampling for GALAH DR3

## Author: Sven Buder

In [ ]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table, hstack, vstack
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [ ]:
debug = True

### Galpy initialization

In [ ]:
import galpy
from galpy.potential import MWPotential2014 as pot
from galpy.actionAngle import actionAngleStaeckel
from galpy.util import bovy_coords

#galpy scale units:                                                                                                                                                                                         
_REFR0 = 8.2   #[kpc]  --> galpy length unit                                                                                                                                                                 
_REFV0 = 238. #[km/s] --> galpy velocity unit

aAS = actionAngleStaeckel(
        pot   = pot,        #potential                                                                                                                                                                      
        delta = 0.45,       #focal length of confocal coordinate system                                                                                                                            
        c     = True        #use C code (for speed)                                                                                                                                                         
        )

print("The Sun has an angular momentum of '+str(_REFR0 * _REFV0)+' kpc km/s in this action framework with MWPotential2014")

#Galactocentric position of the Sun according to Bland-Hawthorn & Gerhard (2016)
X_gc_sun_kpc = 8.2   #[kpc]                                                                                                                                                                                 
Z_gc_sun_kpc = 0.025 #[kpc]                                                                                                                                                                                 

print(r"We place Sgr A at (x; y; z) = (R_0; 0; z_0) kpc, where")
print("R_0 = 8.2 kpc and z_0 = -25 pc (Bland-Hawthorn & Gerhard, 2016)")

#Galactocentric velocity of the Sun:                                                                                                                                                                        
vX_gc_sun_kms = -11.0        # = -U              [kms]                                                                                                                                                      
vY_gc_sun_kms =  10.0+_REFV0 # = V+v_circ(R_Sun) [kms]                                                                                                                                                      
vZ_gc_sun_kms =   7.25       # = W               [kms]                                                                                                                                                      

print("The Sun's velocity with respect to a co-located particle on a circular orbit is")
print("V_LSR = (U_sun, V_sun, W_sun) = (-11, 10, 7.25) km/s (Schoenrich 2012)")

### Input of 6D information in observable dimensions

In [ ]:
# Where is sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis.fits?

file_directory = '/shared-storage/buder/svn-repos/trunk/GALAH/DATA/'
#file_directory = '/Users/buder/GALAH_DR3/input/'

In [ ]:
galah_gaia_input = pyfits.getdata(file_directory+'sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis.fits',1)
full_length = len(galah_gaia_input['sobject_id'])
print(full_length)

subset_size = 25000

try:
    subset = int(sys.argv[1])
except:
    subset = 0
if subset*subset_size >= full_length:
    sys.exit('The subset is beyond the length of GALAH DR3')

galah_gaia_input = galah_gaia_input[subset*subset_size:np.min([(subset+1)*subset_size,full_length])]

nr_galah_stars = len(galah_gaia_input['sobject_id'])
print(nr_galah_stars)

nr_galah_stars_dynamics = np.where(
    np.isfinite(galah_gaia_input['ra']) &
    np.isfinite(galah_gaia_input['dec']) &
    np.isfinite(galah_gaia_input['r_est']) &
    np.isfinite(galah_gaia_input['pmra']) &
    np.isfinite(galah_gaia_input['pmdec']) &
    np.isfinite(galah_gaia_input['rv_guess']) &
    np.isfinite(galah_gaia_input['ra_error']) &
    np.isfinite(galah_gaia_input['dec_error']) &
    np.isfinite(galah_gaia_input['r_hi']) &
    np.isfinite(galah_gaia_input['r_lo']) &
    np.isfinite(galah_gaia_input['pmra_error']) &
    np.isfinite(galah_gaia_input['pmdec_error']) &
    np.isfinite(galah_gaia_input['e_rv_guess']) &
    (galah_gaia_input['rv_guess'] != 999.) &
    (galah_gaia_input['rv_guess'] != 1024.)
    )[0]

# This should only me activated for tests with subsets of GALAH DR3
# nr_galah_stars_dynamics = nr_galah_stars_dynamics[:200]

galah_gaia = galah_gaia_input[nr_galah_stars_dynamics]
nr_stars = len(galah_gaia['sobject_id'])
print(nr_stars)

In [ ]:
six_dimensions = {}

# Right ascension [rad]
six_dimensions['ra'] = galah_gaia['ra']*(np.pi/180.)
# Declination [rad]
six_dimensions['dec'] = galah_gaia['dec']*(np.pi/180.)
# Distance from Sun [kpc]
six_dimensions['distance'] = galah_gaia['r_est']#/1000.

# Total proper motion in direction of right ascension [mas/yr]
six_dimensions['pmra'] = galah_gaia['pmra'] * np.cos(six_dimensions['dec'])
# Total proper motion in direction of declination [mas/yr]
six_dimensions['pmdec'] = galah_gaia['pmdec']
# Radial velocity [km/s]
six_dimensions['vrad'] = galah_gaia['rv_guess']

In [ ]:
e_six_dimensions = {}

# Error of right ascension [rad]
e_six_dimensions['ra'] = galah_gaia['ra_error']*(np.pi/180.)
# Error of declination [rad]
e_six_dimensions['dec'] = galah_gaia['dec_error']*(np.pi/180.)
# Error of distance from Sun [pc]
e_six_dimensions['distance_high'] = galah_gaia['r_hi']
e_six_dimensions['distance_low']  = galah_gaia['r_lo']
    # We are currently sampling a 2-sided Gaussian because Bailer-Jones are only giving 16th/50th/86th percentiles.
    # Any idea how to improve that because of missing posteriors from Bailer-Jones?
    
# Error of total proper motion in direction of right ascension [mas/yr]
e_six_dimensions['pmra'] = galah_gaia['pmra_error'] * np.cos(six_dimensions['dec'])
# Error of total proper motion in direction of declination [mas/yr]
e_six_dimensions['pmdec'] = galah_gaia['pmdec_error']
# Error of radial velocity [km/s]
e_six_dimensions['vrad'] = np.sqrt(galah_gaia['e_rv_guess']**2. + 0.1**2.) # error floor for rv_guess is 0.1 like for GALAH DR2

## Monte Carlo sampling of Orbits

In [ ]:
MC_size = 10000
np.random.seed(123)

XYZ_labels       = ['X_XYZ','Y_XYZ','Z_XYZ']
UVW_labels       = ['U_LSR','V_LSR','W_LSR']

Rphiz_labels     = ['R_Rzphi','phi_Rzphi','z_Rzphi']
vRphiz_labels    = ['vR_Rzphi','vphi_Rzphi','vz_Rzphi']

action_labels    = ['J_R','L_Z','J_Z']
ext_orbit_labels = ['ecc', 'zmax', 'R_peri', 'R_ap']

orbit_labels = np.concatenate((
    XYZ_labels,
    UVW_labels,
    Rphiz_labels,
    vRphiz_labels,
    action_labels,
    ext_orbit_labels    
    ))
print(orbit_labels)

orbit_information = {}
interim_orbit_information = {}

for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = []
    interim_orbit_information[each_orbit_label+'_sig'] = []

### Samples

In [ ]:
# Sampling the distances from Bailer-Jones assuming 2 separate Gaussian distributions
distance_sigma_lo  = np.array([np.abs(np.random.normal(loc = 0., scale = six_dimensions['distance'] -  e_six_dimensions['distance_low'])) for i in range(MC_size)])
distance_sigma_hi  = np.array([np.abs(np.random.normal(loc = 0., scale = e_six_dimensions['distance_high'] - six_dimensions['distance'])) for i in range(MC_size)])
select_lo_hi = np.array([(np.random.uniform(0, 1, size=nr_stars) < 0.5).astype(float) for x in range(MC_size)])

In [ ]:
MC_sample_6D = {}

if MC_size == 1:
    for each_key in six_dimensions.keys():
        MC_sample_6D[each_key] = np.array([six_dimensions[each_key]])
else:
    MC_sample_6D['ra']       = np.array([np.random.normal(loc=six_dimensions['ra'], scale=e_six_dimensions['ra']) for i in range(MC_size)])
    MC_sample_6D['dec']      = np.array([np.random.normal(loc=six_dimensions['dec'], scale=e_six_dimensions['dec']) for i in range(MC_size)])
    MC_sample_6D['distance'] = (six_dimensions['distance'] + select_lo_hi*distance_sigma_hi - (1-select_lo_hi)*distance_sigma_lo).clip(min=0)/1000.

    MC_sample_6D['pmra']       = np.array([np.random.normal(loc=six_dimensions['pmra'], scale=e_six_dimensions['pmra']) for i in range(MC_size)])
    MC_sample_6D['pmdec']      = np.array([np.random.normal(loc=six_dimensions['pmdec'], scale=e_six_dimensions['pmdec']) for i in range(MC_size)])
    MC_sample_6D['vrad']      = np.array([np.random.normal(loc=six_dimensions['vrad'], scale=e_six_dimensions['vrad']) for i in range(MC_size)])

In [ ]:
if (debug == True):
    star_index = 0

    kwargs = dict(bins=50,rasterized=True)
    
    f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3,figsize=(15,5))

    ax1.hist(MC_sample_6D['ra'][:,star_index],**kwargs);
    ax2.hist(MC_sample_6D['dec'][:,star_index],**kwargs);
    ax3.hist(MC_sample_6D['distance'][:,star_index],**kwargs);

    ax4.hist(MC_sample_6D['pmra'][:,star_index],**kwargs);
    ax5.hist(MC_sample_6D['pmdec'][:,star_index],**kwargs);
    ax6.hist(MC_sample_6D['vrad'][:,star_index],**kwargs);

    ax1.set_xlabel('MC (RA) [rad]')
    ax2.set_xlabel('MC (Dec) [rad]')
    ax3.set_xlabel('MC (Dist) [kpc]')
    ax4.set_xlabel('MC (pmra) [mas/yr]')
    ax5.set_xlabel('MC (pmdec) [mas/yr]')
    ax6.set_xlabel('MC (vrad) [km/s]')

    plt.tight_layout()
    plt.savefig('figures/MC_input.png',dpi=300)

In [ ]:
for each_draw in range(MC_size):
    
    #a. convert spatial coordinates (ra,dec,d) to (R,z,phi)
    
    #(ra,dec) --> Galactic coordinates (l,b):  
    lb = bovy_coords.radec_to_lb(
                    MC_sample_6D['ra'][each_draw],MC_sample_6D['dec'][each_draw],
                    degree=False,epoch='J2015.5'
                    )
    
    # (l,b,d) --> Galactocentric cartesian coordinates (x,y,z):  
    xyz = bovy_coords.lbd_to_XYZ(
                    lb[:,0],lb[:,1],
                    MC_sample_6D['distance'][each_draw],
                    degree=False)
    
    interim_orbit_information[XYZ_labels[0]].append(xyz[:,0])
    interim_orbit_information[XYZ_labels[1]].append(xyz[:,1])
    interim_orbit_information[XYZ_labels[2]].append(xyz[:,2])
    
    # (x,y,z) --> Galactocentric cylindrical coordinates (R,phi,z):
    Rphiz= bovy_coords.XYZ_to_galcencyl(
                    xyz[:,0], xyz[:,1], xyz[:,2],
                    Xsun=X_gc_sun_kpc,Zsun=Z_gc_sun_kpc
                    )
    
    #b. convert velocities (pm_ra,pm_dec,vlos) to (vR,vz,vT)     
    pmlpmb = bovy_coords.pmrapmdec_to_pmllpmbb(
                        MC_sample_6D['pmra'][each_draw],
                        MC_sample_6D['pmdec'][each_draw],
                        MC_sample_6D['ra'][each_draw],
                        MC_sample_6D['dec'][each_draw],
                        degree=False,epoch='J2015.5'
                        )
    
    uvw = bovy_coords.vrpmllpmbb_to_vxvyvz(
                    MC_sample_6D['vrad'][each_draw],
                    pmlpmb[:,0],pmlpmb[:,1],
                    lb[:,0],lb[:,1],
                    MC_sample_6D['distance'][each_draw],
                    XYZ=False,degree=False
                    )
    
    interim_orbit_information[Rphiz_labels[0]].append(Rphiz[:,0])
    interim_orbit_information[Rphiz_labels[1]].append(Rphiz[:,1])
    interim_orbit_information[Rphiz_labels[2]].append(Rphiz[:,2])
    
    interim_orbit_information[UVW_labels[0]].append(uvw[:,0])
    interim_orbit_information[UVW_labels[1]].append(uvw[:,1])
    interim_orbit_information[UVW_labels[2]].append(uvw[:,2])

    vRphiz = bovy_coords.vxvyvz_to_galcencyl(
                    uvw[:,0],
                    uvw[:,1],
                    uvw[:,2],
                    Rphiz[:,0],
                    Rphiz[:,1],
                    Rphiz[:,2],
                    vsun=[vX_gc_sun_kms,vY_gc_sun_kms,vZ_gc_sun_kms],
                    galcen=True
                    )
    
    interim_orbit_information[vRphiz_labels[0]].append(vRphiz[:,0])
    interim_orbit_information[vRphiz_labels[1]].append(vRphiz[:,1])
    interim_orbit_information[vRphiz_labels[2]].append(vRphiz[:,2])
    

    jR,lz,jz = aAS(
        Rphiz[:,0]/_REFR0,
        vRphiz[:,0]/_REFV0,
        vRphiz[:,1]/_REFV0,
        Rphiz[:,2]/_REFR0,
        vRphiz[:,2]/_REFV0
    )
    
    interim_orbit_information[action_labels[0]].append(jR*_REFR0*_REFV0)
    interim_orbit_information[action_labels[1]].append(lz*_REFR0*_REFV0)
    interim_orbit_information[action_labels[2]].append(jz*_REFR0*_REFV0)
    
    ezr1, ezr2, ezr3, ezr4 = aAS.EccZmaxRperiRap(
        Rphiz[:,0]/_REFR0,
        vRphiz[:,0]/_REFV0,
        vRphiz[:,1]/_REFV0,
        Rphiz[:,2]/_REFR0,
        vRphiz[:,2]/_REFV0,
        Rphiz[:,1]
    )
    
    interim_orbit_information[ext_orbit_labels[0]].append(ezr1)
    interim_orbit_information[ext_orbit_labels[1]].append(ezr2*_REFR0)
    interim_orbit_information[ext_orbit_labels[2]].append(ezr3*_REFR0)
    interim_orbit_information[ext_orbit_labels[3]].append(ezr4*_REFR0)
    

In [ ]:
for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = np.array(interim_orbit_information[each_orbit_label])
    try:
        orbit_information[each_orbit_label] = np.array([np.nanmean(interim_orbit_information[each_orbit_label][:,each_star]) for each_star in range(nr_stars)])
        orbit_information[each_orbit_label+'_sig'] = np.array([np.nanstd(interim_orbit_information[each_orbit_label][:,each_star]) for each_star in range(nr_stars)])
    except:
        print('no output for '+each_orbit_label)

if debug==True:
    
    star_index = 0
    
    print("XYZ = ({x:8.2f},{y:8.2f},{z:8.2f}) [kpc]".format(
        x=orbit_information[XYZ_labels[0]][star_index],
        y=orbit_information[XYZ_labels[1]][star_index],
        z=orbit_information[XYZ_labels[2]][star_index]
        ))
          
    print("UVW = ({u:8.2f},{v:8.2f},{w:8.2f}) [kpc km/s]".format(
        u=orbit_information[UVW_labels[0]][star_index],
        v=orbit_information[UVW_labels[1]][star_index],
        w=orbit_information[UVW_labels[2]][star_index]
        ))

    print("R   = {r:6.2f} +- {r_sig:6.2f} [kpc], phi = {phi:6.2f} +- {phi_sig:6.2f} [rad], z = {z:6.2f} +- {z_sig:6.2f} [kpc]".format(
        r=orbit_information[Rphiz_labels[0]][star_index],
        phi=orbit_information[Rphiz_labels[1]][star_index],
        z=orbit_information[Rphiz_labels[2]][star_index],
        r_sig=orbit_information[Rphiz_labels[0]+'_sig'][star_index],
        phi_sig=orbit_information[Rphiz_labels[1]+'_sig'][star_index],
        z_sig=orbit_information[Rphiz_labels[2]+'_sig'][star_index]
        ))
    print("J_R = {jr:6.2f} +- {jr_sig:6.2f}, L_Z = {lz:6.2f} +- {lz_sig:6.2f}, J_Z = {jz:6.2f} +- {jz_sig:6.2f} [kpc km/s]".format(
        jr=orbit_information['J_R'][star_index],
        jr_sig=orbit_information['J_R_sig'][star_index],
        lz=orbit_information['L_Z'][star_index],
        lz_sig=orbit_information['L_Z_sig'][star_index],
        jz=orbit_information['J_Z'][star_index],
        jz_sig=orbit_information['J_Z_sig'][star_index]
        ))
    print("e = {ecc:6.2f} +- {ecc_sig:6.2f}, zmax = {zmax:6.2f} +- {zmax_sig:6.2f}, Rperi = {rperi:6.2f} +- {rperi_sig:6.2f} [kpc km/s], Rapo = {rapo:6.2f} +- {rapo_sig:6.2f} [kpc km/s]".format(
        ecc=orbit_information['ecc'][star_index],
        ecc_sig=orbit_information['ecc_sig'][star_index],
        zmax=orbit_information['zmax'][star_index],
        zmax_sig=orbit_information['zmax'][star_index],
        rperi=orbit_information['R_peri'][star_index],
        rperi_sig=orbit_information['R_peri_sig'][star_index],
        rapo=orbit_information['R_ap'][star_index],
        rapo_sig=orbit_information['R_ap_sig'][star_index]
        ))

In [ ]:
if debug==True:
    
    errorbar_kwargs = dict(fmt='o',rasterized=True)
    
    f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3,figsize=(15,10))

    ax1.errorbar(
        orbit_information[XYZ_labels[0]],
        orbit_information[XYZ_labels[1]],
        xerr=orbit_information[XYZ_labels[0]+'_sig'],
        yerr=orbit_information[XYZ_labels[1]+'_sig'],
        **errorbar_kwargs
    )
    ax1.set_xlabel('X (XYZ) [kpc]')
    ax1.set_ylabel('Y (XYZ) [kpc]')
    
    ax2.errorbar(
        orbit_information[Rphiz_labels[0]],
        orbit_information[Rphiz_labels[2]],
        xerr=orbit_information['R_Rzphi_sig'],
        yerr=orbit_information['z_Rzphi_sig'],
        **errorbar_kwargs
    )
    ax2.set_xlabel('R (GC) [kpc]')
    ax2.set_ylabel('z (GC) [kpc]')

    ax3.errorbar(
        orbit_information[UVW_labels[1]],
        np.sqrt(orbit_information[UVW_labels[0]]**2 + orbit_information[UVW_labels[2]]**2),
        xerr=orbit_information[UVW_labels[1]+'_sig'],
        yerr=np.sqrt(orbit_information[UVW_labels[0]+'_sig']**2 + orbit_information[UVW_labels[2]+'_sig']**2),
        **errorbar_kwargs
    )
    ax3.set_xlabel('Toomre V (LSR) [km/s]')
    ax3.set_ylabel('Toomre UW (LSR) [km/s]')

    ax4.errorbar(
        orbit_information[ext_orbit_labels[0]],
        orbit_information[ext_orbit_labels[1]],
        xerr=orbit_information[ext_orbit_labels[0]+'_sig'],
        yerr=orbit_information[ext_orbit_labels[1]+'_sig'],
        **errorbar_kwargs
    )
    ax4.set_xlabel('Eccentricity')
    ax4.set_ylabel(r'$z_\text{max}$ [kpc]')

    ax5.errorbar(
        orbit_information[ext_orbit_labels[2]],
        orbit_information[ext_orbit_labels[3]],
        xerr=orbit_information[ext_orbit_labels[2]+'_sig'],
        yerr=orbit_information[ext_orbit_labels[3]+'_sig'],
        **errorbar_kwargs
    )
    ax5.set_xlabel(r'R (pericenter) [kpc]')
    ax5.set_ylabel(r'R (apocenter) [kpc]')

    ax6.errorbar(
        orbit_information[action_labels[1]],
        np.sqrt(orbit_information[action_labels[0]]),
        xerr=orbit_information[action_labels[1]+'_sig'],
        yerr=[np.sqrt(orbit_information[action_labels[0]]+orbit_information[action_labels[0]+'_sig']) - np.sqrt(orbit_information[action_labels[0]]),np.sqrt(orbit_information[action_labels[0]]) - np.sqrt(orbit_information[action_labels[0]]-orbit_information[action_labels[0]+'_sig'])],
        **errorbar_kwargs
    )
    ax6.set_xlabel(r'$L_Z$ [kpc km/s]')
    ax6.set_ylabel(r'sqrt($J_R$ [kpc km/s])')
    ax6.set_ylim(-10,100)
    ax6.set_xlim(-1000,5000)
    
    plt.tight_layout()
    
    plt.savefig('figures/MC_output.png',dpi=300)

In [ ]:
galah_dynamics = {}

galah_dynamics['sobject_id'] = galah_gaia_input['sobject_id']
for each_orbit_label in orbit_labels:
    galah_dynamics[each_orbit_label] = np.zeros(nr_galah_stars, dtype=float)
    galah_dynamics[each_orbit_label].fill(np.nan)
    galah_dynamics[each_orbit_label+'_sig'] = np.zeros(nr_galah_stars, dtype=float)
    galah_dynamics[each_orbit_label+'_sig'].fill(np.nan)

    (galah_dynamics[each_orbit_label])[nr_galah_stars_dynamics] = orbit_information[each_orbit_label]
    (galah_dynamics[each_orbit_label+'_sig'])[nr_galah_stars_dynamics] = orbit_information[each_orbit_label+'_sig']

galah_dynamics_data = pandas.DataFrame(galah_dynamics,columns=galah_dynamics.keys())
galah_dynamics_data

data_for_fits = Table.from_pandas(galah_dynamics_data)
data_for_fits.write('sobject_dynamics.fits',overwrite=True)

In [ ]:
try:
    import email
    import email.mime.application
    from email.MIMEMultipart import MIMEMultipart
    from email.MIMEText import MIMEText
    from email.MIMEImage import MIMEImage
    msg = MIMEMultipart()

    msg['From'] = 'gemini2'
    msg['To'] = 'buder@mpia.de'
    msg['Subject'] = 'Orbit calculation for GALAH DR3 finished'                                                                                                         

    import smtplib
    mailer = smtplib.SMTP('localhost')
    mailer.sendmail('gemini2', 'buder@mpia.de', msg.as_string())
    mailer.close()
    print('Email sent')
except:
    print('Could not send email')